In [1]:
import pandas as pd

In [2]:
matches = pd.read_excel('football_data.xlsx')


FileNotFoundError: [Errno 2] No such file or directory: 'football_data.xlsx'

In [ ]:
class MisssingDict(dict):
    __missing__ = lambda self, key:key
    
map_values = {
    "Brighton and Hove Albion": "Brighton",
    "Manchester United": "Manchester Utd",
    "Tottenham Hotspur": "Tottenham",
    "West Ham United": "West Ham",
    "Wolverhampton Wanderers": "Wolves",
    "Huddersfield Town" : "Huddersfield",
    "West Bromwich Albion": "West Brom",
    'Nottingham Forest': "Nott'ham Forest",
    'Sheffield United': 'Sheffield Utd',
    'Newcastle United' :'Newcastle Utd'
}

mapping = MisssingDict(map_values)

#Map the value in Team column to match the name of Opponent columns

In [ ]:
only_in_a = set(matches['Team']) - set(matches['Opponent'])
only_in_b = set(matches['Opponent']) - set(matches['Team'])

In [ ]:
matches["Team"] = matches["Team"].map(mapping)

In [ ]:
matches["Round"].value_counts() # week 37 38 missing 2 rows (at current time of writing) since they are not played yet

Round
Matchweek 1     200
Matchweek 2     200
Matchweek 3     200
Matchweek 4     200
Matchweek 5     200
Matchweek 6     200
Matchweek 7     200
Matchweek 8     200
Matchweek 9     200
Matchweek 10    200
Matchweek 11    200
Matchweek 12    200
Matchweek 13    200
Matchweek 14    200
Matchweek 16    200
Matchweek 17    200
Matchweek 18    200
Matchweek 19    200
Matchweek 20    200
Matchweek 21    200
Matchweek 22    200
Matchweek 23    200
Matchweek 24    200
Matchweek 15    200
Matchweek 25    200
Matchweek 29    200
Matchweek 26    200
Matchweek 27    200
Matchweek 28    200
Matchweek 30    200
Matchweek 31    200
Matchweek 32    200
Matchweek 33    200
Matchweek 34    200
Matchweek 35    200
Matchweek 36    200
Matchweek 37    180
Matchweek 38    180
Name: count, dtype: int64

In [ ]:
matches['Date'] = pd.to_datetime(matches['Date'])

In [ ]:
matches["Venue_Code"]= matches["Venue"].astype('category').cat.codes

In [ ]:
matches = matches.sort_values(["Team", "Season", "Date"], ascending=[True, True, True])

In [ ]:
matches.index = range(matches.shape[0])

In [ ]:
columns_to_average = ["Sh", "SoT", "Dist", "GF", "GA","xG", "xGA", "Poss" ]

In [ ]:
rolling_columns = [f"{col}_rolling_5" for col in columns_to_average]
rolling_columns

['Sh_rolling_5',
 'SoT_rolling_5',
 'Dist_rolling_5',
 'GF_rolling_5',
 'GA_rolling_5',
 'xG_rolling_5',
 'xGA_rolling_5',
 'Poss_rolling_5']

In [ ]:
for col, rolling_col in zip(columns_to_average, rolling_columns):
    matches[rolling_col] = matches.groupby(["Team", "Season"])[col].rolling(5, closed = "left").mean().reset_index(level=[0,1], drop=True)


In [ ]:
team_rolling_stats = matches[["Opponent", "Season", "Date"] + rolling_columns].copy()
team_rolling_stats = team_rolling_stats.rename(columns = {col: f"Opponent_{col}" for col in rolling_columns})

In [ ]:
team_rolling_stats

,Opponent,Season,Date,Opponent_Sh_rolling_5,Opponent_SoT_rolling_5,Opponent_Dist_rolling_5,Opponent_GF_rolling_5,Opponent_GA_rolling_5,Opponent_xG_rolling_5,Opponent_xGA_rolling_5,Opponent_Poss_rolling_5
0,West Ham,2016,2015-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Crystal Palace,2016,2015-08-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Liverpool,2016,2015-08-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Newcastle Utd,2016,2015-08-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Stoke City,2016,2015-09-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
7555,Tottenham,2025,2025-04-13,13.0,3.8,17.88,1.4,1.0,1.26,1.26,52.6
7556,Manchester Utd,2025,2025-04-20,12.0,3.8,17.40,2.0,1.0,1.48,1.32,48.0
7557,Leicester City,2025,2025-04-26,10.6,3.6,17.86,2.0,0.8,1.36,1.34,43.0
7558,Manchester City,2025,2025-05-02,13.6,4.0,18.12,2.2,0.6,1.68,1.30,46.2


In [ ]:
matches = matches.merge(team_rolling_stats,
                          left_on = ["Team", "Season", "Date"],
                          right_on=["Opponent", "Season", "Date"], 
                          suffixes=("", "_opponent")#avoid column name conflicts
) 

In [ ]:
matches.drop(columns = "Opponent_opponent", inplace = True)

In [ ]:
#Step3: compute the difference between home'team and opponent's rolling averages
difference_columns = [f"{col}_diff" for col in columns_to_average]
for col, diff_col in zip(rolling_columns, difference_columns):
    matches[diff_col] = matches[col]-matches[f"Opponent_{col}"]


In [ ]:
matches["Opp_Code"] = matches["Opponent"].astype('category').cat.codes

In [ ]:
matches["Hour"] = matches["Time"].str.extract(r"(\d{2}:\d{2})")

In [ ]:
matches["Hour"] = matches["Hour"].str.replace(r":.+", "", regex=True).astype(int)

In [ ]:
matches["day_code"] = matches["Date"].dt.dayofweek

In [ ]:
matches["target"] = (matches["Result"] == "W").astype(int)

In [ ]:
matches["is_unbeaten"] = matches["Result"].isin(["W", "D"])

In [ ]:
matches["is_unbeaten_shifted"] = matches.groupby(["Team", "Season"])["is_unbeaten"].shift(1)
#shift the is_unbeaten column to exclude the current match
#Note: the first match of each group will have a NaN value in the shifted column(which is treated as False in the streak calculation)

In [ ]:
import numpy as np

In [ ]:
def compute_unbeaten_streak(group):
    streak = 0
    streaks = []
    for unbeaten in group["is_unbeaten_shifted"]:
        if(unbeaten is np.nan):
            streak = 0
        elif(unbeaten):
            streak +=1
        else:
            streak = 0 # reset streak
        streaks.append(streak)
    return streaks


matches["unbeaten_streak"] = (
    matches.groupby(["Team", "Season"])
    .apply(lambda x: pd.Series(compute_unbeaten_streak(x), index = x.index))
    .reset_index(level = [0,1], drop = True) #align with original dataframe
)

matches = matches.drop(columns= ["is_unbeaten_shifted"])



C:\Users\This PC\AppData\Local\Temp\ipykernel_28136\3674810700.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series(compute_unbeaten_streak(x), index = x.index))


In [ ]:
matches.rename(columns = {"unbeaten_streak": "unbeaten_streak_home"}, inplace=True)

In [ ]:
unbeaten_streak_temp = matches[["Opponent", "Season", "Date", "unbeaten_streak_home"]].copy()

In [ ]:
unbeaten_streak_temp.rename(columns = {"unbeaten_streak_home" : "unbeaten_streak_away"}, inplace =  True)

In [ ]:
unbeaten_streak_temp

,Opponent,Season,Date,unbeaten_streak_away
0,West Ham,2016,2015-08-09,0
1,Crystal Palace,2016,2015-08-16,0
2,Liverpool,2016,2015-08-24,1
3,Newcastle Utd,2016,2015-08-29,2
4,Stoke City,2016,2015-09-12,3
...,...,...,...,...
7555,Tottenham,2025,2025-04-13,4
7556,Manchester Utd,2025,2025-04-20,5
7557,Leicester City,2025,2025-04-26,6
7558,Manchester City,2025,2025-05-02,7


In [ ]:
matches = matches.merge(unbeaten_streak_temp,
                          left_on = ["Team", "Season", "Date"],
                          right_on=["Opponent", "Season", "Date"], 
                          
) 

In [ ]:
matches.drop(columns = "Opponent_y", inplace = True)
matches.rename(columns = {"Opponent_x" : "Opponent"}, inplace= True)

In [ ]:
matches["unbeaten_streak_diff"] = matches["unbeaten_streak_home"] - matches["unbeaten_streak_away"]

In [ ]:
matches

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,xGA_diff,Poss_diff,Opp_Code,Hour,day_code,target,is_unbeaten,unbeaten_streak_home,unbeaten_streak_away,unbeaten_streak_diff
0,2015-08-09,13:30 (19:30),Premier League,Matchweek 1,Sun,Home,L,0,2,West Ham,...,NaN,NaN,32,13,6,0,False,0,0,0
1,2015-08-16,13:30 (19:30),Premier League,Matchweek 2,Sun,Away,W,2,1,Crystal Palace,...,NaN,NaN,8,13,6,1,True,0,1,-1
2,2015-08-24,20:00 (02:00),Premier League,Matchweek 3,Mon,Home,D,0,0,Liverpool,...,NaN,NaN,16,20,0,0,True,1,2,-1
3,2015-08-29,12:45 (18:45),Premier League,Matchweek 4,Sat,Away,W,1,0,Newcastle Utd,...,NaN,NaN,21,12,5,1,True,2,1,1
4,2015-09-12,15:00 (21:00),Premier League,Matchweek 5,Sat,Home,W,2,0,Stoke City,...,NaN,NaN,26,15,5,1,True,3,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7555,2025-04-13,14:00 (20:00),Premier League,Matchweek 32,Sun,Home,W,4,2,Tottenham,...,-0.20,0.8,29,14,6,1,True,4,1,3
7556,2025-04-20,14:00 (20:00),Premier League,Matchweek 33,Sun,Away,W,1,0,Manchester Utd,...,0.16,-0.2,19,14,6,1,True,5,0,5
7557,2025-04-26,15:00 (21:00),Premier League,Matchweek 34,Sat,Home,W,3,0,Leicester City,...,-1.18,-1.4,15,15,5,1,True,6,0,6
7558,2025-05-02,20:00 (02:00),Premier League,Matchweek 35,Fri,Away,L,0,1,Manchester City,...,0.20,-18.8,18,20,4,0,False,7,6,1


In [ ]:
team_ranks = matches.drop_duplicates(subset=["Season", "Team"]).copy() #extract the first occurence of each team in each season()

In [ ]:
team_ranks["Season_Rank"] = team_ranks.groupby("Season").cumcount() + 1
#cumcount() : count starting from 0 for each repated occurence within a group

In [ ]:
#shift rankings to the next season
previous_season_ranks = team_ranks[["Season", "Team", "Season_Rank"]].copy()

In [ ]:
previous_season_ranks["Season"] +=1

In [ ]:
matches = matches.merge(
                        previous_season_ranks,
                        on= ["Season", "Team"],
                        how= "left"
)

In [ ]:
matches.rename(columns={"Season_Rank" : "home_ranking_last_season"}, inplace = True)

In [ ]:
matches["home_ranking_last_season"] = matches["home_ranking_last_season"].fillna(-1) # assigned -1 for promoted team

In [ ]:
earliest_season = matches["Season"].min() #find the earlist season in the amount of data we scrape

In [ ]:
#Define the ranking for the previous season(manually)
previous_season_rankings = {
    "Arsenal": 3,
    "Leicester City": 14,
    "Tottenham": 5,
    "Manchester City": 2,
    "Manchester Utd": 4,
    "Southampton": 7,
    "West Ham": 12,
    "Liverpool": 6,
    "Stoke City": 9,
    "Chelsea": 1,
    "Everton": 11,
    "Swansea City": 8,
    "Watford": -1,
    "West Brom": 13,
    "Crystal Palace": 10,
    "Bournemouth": -1,
    "Sunderland": 16,
    "Newcastle Utd": 15,
    "Norwich City": -1,
    "Aston Villa": 17
}

In [ ]:
matches.loc[matches["Season"] == earliest_season, "home_ranking_last_season"] = (
    matches.loc[matches["Season"] == earliest_season, "Team"].map(previous_season_rankings)
)

In [ ]:
matches["home_ranking_last_season"] = matches["home_ranking_last_season"].astype(int)

In [ ]:
previous_season_ranks = matches[["Opponent", "Season", "Date", "home_ranking_last_season"]].copy()

In [ ]:
previous_season_ranks.rename(columns = {"home_ranking_last_season": "away_ranking_last_season"}, inplace = True)

In [ ]:
matches = matches.merge(previous_season_ranks,
                          left_on = ["Team", "Season", "Date"],
                          right_on=["Opponent", "Season", "Date"], 
                          
) 

In [ ]:
matches.rename(columns={"Opponent_x" : "Opponent"}, inplace=True)
matches.drop(columns = ["Opponent_y"], inplace = True)

In [ ]:
matches["LastSeasonRank"] = matches["home_ranking_last_season"] - matches["away_ranking_last_season"]

In [ ]:
#Create the PromotedMatchup column based on whether they are recently promoted or not
matches["PromotedMatchup"] = 0 # Default value: Both teams are not recently promoted or both are promoted

In [ ]:
#Condition 1: Home team is promoted, away is not
matches.loc[(matches["home_ranking_last_season"]) == -1 & (matches["away_ranking_last_season"] != -1), "PromotedMatchup"] = 1

In [ ]:
# Condition 2: Away team is promoted, home team is not
matches.loc[
    (matches["away_ranking_last_season"] == -1) & (matches["home_ranking_last_season"] != -1),
    "PromotedMatchup"
] = -1

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [ ]:
mid_season_2025 = matches[(matches["Season"] == 2025) & (matches["Round"].str.contains("19"))]["Date"].min()

In [ ]:
rf = RandomForestClassifier(n_estimators=100, max_depth = 10, random_state=1, min_samples_split=5)

In [ ]:
def make_predictions(data, predictors):
    train = data[data["Date"] < mid_season_2025]
    test = data[data["Date"] > mid_season_2025]
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame({"Actual": test["target"], "Predicted": preds}, index=test.index)
    precision = precision_score(test["target"], preds)
    accuracy = accuracy_score(test["target"], preds)
    return combined, precision, accuracy


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

def make_predictions_stratified(data, predictors):
    # Stratified split for train and test sets
    stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1)
    for train_index, test_index in stratified_split.split(data, data["target"]):
        train = data.iloc[train_index]
        test = data.iloc[test_index]
    
    # Train the model on the stratified training set
    rf.fit(train[predictors], train["target"])
    
    # Make predictions on the test set
    preds = rf.predict(test[predictors])
    
    # Evaluate the model
    combined = pd.DataFrame({"Actual": test["target"], "Predicted": preds}, index=test.index)
    precision = precision_score(test["target"], preds)
    accuracy = accuracy_score(test["target"], preds)
    
    return combined, precision, accuracy

In [ ]:
from sklearn.utils import resample
import pandas as pd

def make_predictions_balanced(data, predictors):
    # Separate majority and minority classes
    class_counts = data["target"].value_counts()
    majority_class = class_counts.idxmax()
    minority_class = class_counts.idxmin()
    
    df_majority = data[data["target"] == majority_class]
    df_minority = data[data["target"] == minority_class]

    # Undersample majority class to match minority class
    df_majority_downsampled = resample(
        df_majority,
        replace=False,
        n_samples=len(df_minority),
        random_state=1
    )

    # Combine to get balanced data
    balanced_data = pd.concat([df_majority_downsampled, df_minority])

    # Shuffle the combined balanced dataset
    balanced_data = balanced_data.sample(frac=1, random_state=1).reset_index(drop=True)

    # Split into train and test (random split)
    train = balanced_data.sample(frac=0.8, random_state=1)
    test = balanced_data.drop(train.index)

    # Train the model
    rf.fit(train[predictors], train["target"])

    # Predict on test set
    preds = rf.predict(test[predictors])

    # Evaluate
    combined = pd.DataFrame({"Actual": test["target"], "Predicted": preds}, index=test.index)
    precision = precision_score(test["target"], preds)
    accuracy = accuracy_score(test["target"], preds)

    return combined, precision, accuracy


In [ ]:
predictors = ['SoT_diff', 'Poss_diff', 'unbeaten_streak_diff',"GA_diff","GF_diff",'xG_diff', 'xGA_diff',
       'LastSeasonRank', 'PromotedMatchup']

In [ ]:
matchesB = matches[["Round","Season", "Date", "Team","Opponent", "Venue",  "Result", 'Sh_diff', 'SoT_diff', 'GF_diff', 'GA_diff', 'xG_diff', 'xGA_diff', 'Poss_diff', 'unbeaten_streak_diff',
       'LastSeasonRank', 'PromotedMatchup',"target"]]

In [ ]:
matchesB = matchesB[matchesB["Venue"] == "Home"]

In [ ]:
matchesB.dropna(subset = ['Sh_diff', 'SoT_diff', 'GF_diff', 'GA_diff', 'xG_diff', 'xGA_diff', 'Poss_diff'], inplace=True)

In [ ]:
#fine_tune hyper parameter:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from scipy.stats import randint
import pandas as pd

#Step1: Data Preparation

X = matchesB[predictors]
y = matchesB["target"]


#Split data into Training, Validation, and Test Sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify= y  )

#Step 3: Define the Hyperparameter Space

param_dist = {
    'n_estimators': randint(100, 300), # Number of trees
    'max_depth': [None] + list(range(5,31,5)), # Tree depth
    'min_samples_split': randint(2,10),
    'min_samples_leaf': randint(1,5),
    'max_features' : ['sqrt', 'log2'],
    'bootstrap' : [True]
}


#Step4: Choose Search Straterfy(Random Search) and Cross-Validation
rf = RandomForestClassifier(random_state= 42)
cv_strategy = StratifiedKFold(n_splits = 5, shuffle= True, random_state= 42)

random_search = RandomizedSearchCV(
    estimator = rf,
    param_distributions=param_dist,
    n_iter = 50, #number of parameter combinations to try
    cv = cv_strategy,
    n_jobs = -1, # Use all available cores
    scoring= 'accuracy', #Metric to optimize
    verbose=2,
    random_state=42,
    return_train_score=True
)

#Step5: Perform the Search
random_search.fit(X_train_val, y_train_val)


#Step 6: Analyze Results and Select the Best Model
print("Best Paramters found: ", random_search.best_params_)
print("Best Cross_validation Score: {:.4f}".format(random_search.best_score_))

#Conver cv_result_ to a DataFrame for analysis
cv_results_df = pd.DataFrame(random_search.cv_results_)
print(cv_results_df[['param_n_estimators', 'param_max_depth', 'mean_train_score', 'mean_test_score', 'rank_test_score']].sort_values(by='rank_test_score').head())

best_rf_model = random_search.best_estimator_
y_pred_test = best_rf_model.predict(X_test)

print("\n--- Test Set Evaluation ---")
print("Accuracy:", accuracy_score(y_test, y_pred_test))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Paramters found:  {'bootstrap': True, 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 7, 'n_estimators': 202}
Best Cross_validation Score: 0.6407
    param_n_estimators param_max_depth  mean_train_score  mean_test_score  \
33                 202               5          0.711778         0.640726   
29                 250               5          0.709149         0.639290   
40                 284               5          0.713092         0.638812   
16                 233               5          0.710105         0.637380   
18                 289               5          0.710703         0.637379   

    rank_test_score  
33                1  
29                2  
40                3  
16                4  
18                5  

--- Test Set Evaluation ---
Accuracy: 0.6450381679389313
Confusion Matrix:
 [[222  67]
 [119 116]]
Classification Report:
               precision    recal

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.utils import resample
from scipy.stats import randint
import pandas as pd

# Step 1: Data Preparation
X = matchesB[predictors]
y = matchesB["target"]

# Step 2: Split Data into Training, Validation, and Test Sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 3: Balance the Training Data
train_data = pd.concat([X_train_val, y_train_val], axis=1)

# Separate majority and minority classes
majority_class = train_data[train_data["target"] == 0]
minority_class = train_data[train_data["target"] == 1]

# Upsample the minority class to match the majority class size
minority_upsampled = resample(
    minority_class,
    replace=True,  # Sample with replacement
    n_samples=len(majority_class),  # Match majority class size
    random_state=42  # For reproducibility
)

# Combine the upsampled minority class with the majority class
balanced_train_data = pd.concat([majority_class, minority_upsampled])

# Shuffle the balanced training data
balanced_train_data = balanced_train_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Separate features and target from the balanced training data
X_train_balanced = balanced_train_data[predictors]
y_train_balanced = balanced_train_data["target"]

# Step 4: Define the Hyperparameter Search Space
param_dist = {
    'n_estimators': randint(100, 300),  # Number of trees
    'max_depth': list(range(5, 31, 5)),  # Tree depth
    'min_samples_split': randint(2, 10),  # Minimum samples to split
    'min_samples_leaf': randint(1, 5),  # Minimum samples at leaf
    'max_features': ['sqrt', 'log2'],  # Features to consider for split
    'bootstrap': [True]  # Bootstrap sampling
}

# Step 5: Choose Search Strategy and Cross-Validation
rf = RandomForestClassifier(random_state=42)
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,  # Number of parameter combinations to try
    cv=cv_strategy,
    n_jobs=-1,  # Use all available cores
    scoring='accuracy',  # Metric to optimize
    verbose=2,
    random_state=42,
    return_train_score=True  # To analyze overfitting
)

# Step 6: Perform the Search
random_search.fit(X_train_balanced, y_train_balanced)

# Step 7: Analyze Results and Select the Best Model
print("Best Parameters Found: ", random_search.best_params_)
print("Best Cross-Validation Score: {:.4f}".format(random_search.best_score_))

# Convert cv_results_ to a DataFrame for analysis
cv_results_df = pd.DataFrame(random_search.cv_results_)
print(cv_results_df[['param_n_estimators', 'param_max_depth', 'mean_train_score', 'mean_test_score', 'rank_test_score']].sort_values(by='rank_test_score').head())

# Step 8: Final Evaluation on the Test Set
best_rf_model = random_search.best_estimator_
y_pred_test = best_rf_model.predict(X_test)

print("\n--- Test Set Evaluation ---")
print("Accuracy:", accuracy_score(y_test, y_pred_test))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Parameters Found:  {'bootstrap': True, 'max_depth': 30, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 271}
Best Cross-Validation Score: 0.7968
    param_n_estimators  param_max_depth  mean_train_score  mean_test_score  \
30                 271               30          0.999350         0.796798   
5                  157               20          0.997834         0.785535   
3                  187               15          0.999892         0.784232   
4                  291               20          0.995234         0.783799   
21                 210               25          1.000000         0.783795   

    rank_test_score  
30                1  
5                 2  
3                 3  
4                 4  
21                5  

--- Test Set Evaluation ---
Accuracy: 0.6316793893129771
Confusion Matrix:
 [[229  60]
 [133 102]]
Classification Report:
               precision 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.utils import resample
from scipy.stats import randint
import pandas as pd

# Step 1: Data Preparation
X = matchesB[predictors]
y = matchesB["target"]

# Step 2: Split Data into Training, Validation, and Test Sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 3: Balance the Training Data (Downsampling)
train_data = pd.concat([X_train_val, y_train_val], axis=1)

# Separate majority and minority classes
majority_class = train_data[train_data["target"] == 0]
minority_class = train_data[train_data["target"] == 1]

# Downsample the majority class to match the minority class size
majority_downsampled = resample(
    majority_class,
    replace=False,  # Sample without replacement
    n_samples=len(minority_class),  # Match minority class size
    random_state=42  # For reproducibility
)

# Combine the downsampled majority class with the minority class
balanced_train_data = pd.concat([majority_downsampled, minority_class])

# Shuffle the balanced training data
balanced_train_data = balanced_train_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Separate features and target from the balanced training data
X_train_balanced = balanced_train_data[predictors]
y_train_balanced = balanced_train_data["target"]

# Step 4: Define the Hyperparameter Search Space
param_dist = {
    'n_estimators': randint(100, 300),  # Number of trees
    'max_depth': [None] + list(range(5, 31, 5)),  # Tree depth
    'min_samples_split': randint(2, 10),  # Minimum samples to split
    'min_samples_leaf': randint(1, 5),  # Minimum samples at leaf
    'max_features': ['sqrt', 'log2'],  # Features to consider for split
    'bootstrap': [True]  # Bootstrap sampling
}

# Step 5: Choose Search Strategy and Cross-Validation
rf = RandomForestClassifier(random_state=42)
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,  # Number of parameter combinations to try
    cv=cv_strategy,
    n_jobs=-1,  # Use all available cores
    scoring='accuracy',  # Metric to optimize
    verbose=2,
    random_state=42,
    return_train_score=True  # To analyze overfitting
)

# Step 6: Perform the Search
random_search.fit(X_train_balanced, y_train_balanced)

# Step 7: Analyze Results and Select the Best Model
print("Best Parameters Found: ", random_search.best_params_)
print("Best Cross-Validation Score: {:.4f}".format(random_search.best_score_))


# Convert cv_results_ to a DataFrame for analysis
cv_results_df = pd.DataFrame(random_search.cv_results_)
print(cv_results_df[['param_n_estimators', 'param_max_depth', 'mean_train_score', 'mean_test_score', 'rank_test_score']].sort_values(by='rank_test_score').head())


# Step 8: Final Evaluation on the Test Set
best_rf_model = random_search.best_estimator_
y_pred_test = best_rf_model.predict(X_test)

print("\n--- Test Set Evaluation ---")
print("Accuracy:", accuracy_score(y_test, y_pred_test))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Parameters Found:  {'bootstrap': True, 'max_depth': 5, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 7, 'n_estimators': 284}
Best Cross-Validation Score: 0.6416
    param_n_estimators param_max_depth  mean_train_score  mean_test_score  \
40                 284               5          0.708601         0.641638   
35                 162               5          0.711529         0.640577   
27                 140              10          0.907482         0.640041   
8                  289              30          0.972044         0.639515   
34                 189              10          0.894968         0.638977   

    rank_test_score  
40                1  
35                2  
27                3  
8                 4  
34                5  

--- Test Set Evaluation ---
Accuracy: 0.6145038167938931
Confusion Matrix:
 [[170 119]
 [ 83 152]]
Classification Report:
               precision    reca

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from scipy.stats import randint, uniform
import pandas as pd

# Step 1: Data Preparation
X = matchesB[predictors]
y = matchesB["target"]

# Step 2: Split Data into Training, Validation, and Test Sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 3: Define the Hyperparameter Search Space
param_dist = {
    'n_estimators': randint(100, 300),  # Number of boosting stages
    'learning_rate': uniform(0.01, 0.3),  # Learning rate
    'max_depth': randint(3, 10),  # Maximum depth of the individual estimators
    'min_samples_split': randint(2, 10),  # Minimum samples to split a node
    'min_samples_leaf': randint(1, 5),  # Minimum samples at leaf node
    'subsample': uniform(0.7, 0.3),  # Fraction of samples used for fitting each base learner
    'max_features': ['sqrt', 'log2', None]  # Features to consider for split
}

# Step 4: Choose Search Strategy and Cross-Validation
gb = GradientBoostingClassifier(random_state=42)
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

random_search = RandomizedSearchCV(
    estimator=gb,
    param_distributions=param_dist,
    n_iter=50,  # Number of parameter combinations to try
    cv=cv_strategy,
    n_jobs=-1,  # Use all available cores
    scoring='accuracy',  # Metric to optimize
    verbose=2,
    random_state=42,
    return_train_score=True  # To analyze overfitting
)

# Step 5: Perform the Search
random_search.fit(X_train_val, y_train_val)

# Step 6: Analyze Results and Select the Best Model
print("Best Parameters Found: ", random_search.best_params_)
print("Best Cross-Validation Score: {:.4f}".format(random_search.best_score_))

# Convert cv_results_ to a DataFrame for analysis
cv_results_df = pd.DataFrame(random_search.cv_results_)
print(cv_results_df[['param_n_estimators', 'param_learning_rate', 'param_max_depth', 
                     'mean_train_score', 'mean_test_score', 'rank_test_score']].sort_values(by='rank_test_score').head())

# Step 7: Final Evaluation on the Test Set
best_gb_model = random_search.best_estimator_
y_pred_test = best_gb_model.predict(X_test)

print("\n--- Test Set Evaluation ---")
print("Accuracy:", accuracy_score(y_test, y_pred_test))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Parameters Found:  {'learning_rate': np.float64(0.01469092202235818), 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 9, 'n_estimators': 162, 'subsample': np.float64(0.9446384285364502)}
Best Cross-Validation Score: 0.6388
    param_n_estimators  param_learning_rate  param_max_depth  \
14                 162             0.014691                3   
36                 138             0.038853                3   
48                 257             0.017295                3   
26                 297             0.010156                7   
28                 285             0.060487                3   

    mean_train_score  mean_test_score  rank_test_score  
14          0.689442         0.638814                1  
36          0.706643         0.636897                2  
48          0.701506         0.635942                3  
26          0.906115         0.624951                4  
28  

In [ ]:
matchesB["Round"] = matchesB["Round"].str.extract(r"(\d+)").astype(int)

In [ ]:
matchesB

,Round,Season,Date,Team,Opponent,Venue,Result,Sh_diff,SoT_diff,GF_diff,GA_diff,xG_diff,xGA_diff,Poss_diff,unbeaten_streak_diff,LastSeasonRank,PromotedMatchup,target
81,6,2018,2017-09-25,Arsenal,West Brom,Home,W,5.8,2.8,0.6,0.8,0.70,0.58,21.2,1,-18,1,1
82,7,2018,2017-10-01,Arsenal,Brighton,Home,W,4.0,1.6,0.0,0.0,0.86,0.02,11.4,2,2,-1,1
85,10,2018,2017-10-28,Arsenal,Swansea City,Home,W,8.8,4.8,1.2,-0.2,1.22,-0.18,14.8,1,-15,1,1
87,12,2018,2017-11-18,Arsenal,Tottenham,Home,W,3.2,2.4,0.2,1.2,0.36,0.62,1.8,-1,-16,1,1
89,14,2018,2017-11-29,Arsenal,Huddersfield,Home,W,9.6,3.8,1.4,-0.8,1.26,-0.36,19.4,2,2,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7551,28,2025,2025-03-08,Wolves,Everton,Home,D,1.4,-1.0,-1.2,-0.2,-0.08,0.28,6.8,-7,11,0,0
7553,30,2025,2025-04-01,Wolves,West Ham,Home,W,3.6,1.2,0.4,0.6,0.26,0.24,5.0,1,1,0,1
7555,32,2025,2025-04-13,Wolves,Tottenham,Home,W,2.2,-0.6,0.4,-0.4,-0.08,-0.20,0.8,3,2,0,1
7557,34,2025,2025-04-26,Wolves,Leicester City,Home,W,2.6,1.4,1.6,-1.4,0.68,-1.18,-1.4,6,21,-1,1
